# Method 3

In [ ]:
# Normalizing teh images, With Balancing the data and Standard Scalar normailzation, PCA with various models

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
import shutil
import os
from imblearn.over_sampling import SMOTE
import sys
from sklearn.ensemble import BaggingClassifier
import cv2
import numpy as np
import cv2
import os
from scipy import ndimage
from sklearn.neural_network import MLPClassifier
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
from os import listdir
from matplotlib import image
from natsort import natsorted
import pandas as pd 

In [ ]:
def readImages(direc): #pixel normalixation
    loaded_images = []
    for filename in natsorted(listdir(direc)):
        img_data =Image.open(direc+filename)
        pixels = asarray(img_data)
        pixels = pixels.astype('float32')
        mean = pixels.mean()
        pixels = pixels - mean
        mean = pixels.mean()
        #print('Mean: %.3f' % mean)
        loaded_images.append(pixels.flatten())
    return loaded_images

In [ ]:
X_train=readImages('./bonus-sml-2020/SML_Train/')
y_train=pd.read_csv("./bonus-sml-2020/SML_Train.csv")['category']
y_train=y_train.to_numpy()
X_train=np.asarray(X_train)
print("Lnegth of images:",X_train[0].shape)
print("No of images:",X_train.shape)
print("Train label:",len(y_train))
print(type(X_train))

In [ ]:
    unique,count=np.unique(y_train,return_counts=True) #apllying SMOTE function for data balancing
    temp_dict={k:v for(k,v) in zip(unique,count)}
    print(temp_dict)
    smt = SMOTE(random_state=42)
    X_train, y_train = smt.fit_sample(X_train,y_train)
    unique,count=np.unique(y_train,return_counts=True)
    temp_dict={k:v for(k,v) in zip(unique,count)}
    print(temp_dict)

In [ ]:
# Model 1
#StandardScaler,PCA,SVM on GridCV

constant_filter = StandardScaler()
pca = PCA(n_components=100, whiten=True, random_state=42,svd_solver='randomized')
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(constant_filter,pca,svc)
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005,
                             0.001, 0.005]}
grid = GridSearchCV(model, param_grid)
%time grid.fit(X_train, y_train)

In [ ]:
# Model 2
#StandardScaler,PCA,Baggig SVM

from sklearn.model_selection import GridSearchCV    #StandardScaler,PCA,SVM on GridCV
constant_filter = StandardScaler()
pca = PCA(n_components=100, whiten=True, random_state=42,svd_solver='randomized')
svc = SVC(kernel='rbf', class_weight='balanced',C=5,gamma=0.005)
bag = BaggingClassifier(base_estimator=svc, n_estimators=31, random_state=314)
model = make_pipeline(constant_filter,pca, bag)
%time model.fit(X_train, y_train)

In [ ]:
# Model 3
#StandardScaler,PCA,MLP

from sklearn.model_selection import GridSearchCV    #StandardScaler,PCA,SVM on GridCV
constant_filter = StandardScaler()
pca = PCA(n_components=100, whiten=True, random_state=42,svd_solver='randomized')
svc = SVC(kernel='rbf', class_weight='balanced',C=5,gamma=0.005)
clf = MLPClassifier(activation='relu',solver='adam', alpha=1e-8,
                    hidden_layer_sizes=(32,32,32), random_state=0)
model = make_pipeline(constant_filter,pca, clf)
%time model.fit(X_train, y_train)

In [ ]:
# Model 4
#StandardScaler,PCA,Boosting

from sklearn.model_selection import GridSearchCV    #StandardScaler,PCA,SVM on GridCV
constant_filter = StandardScaler()
pca = PCA(n_components=100, whiten=True, random_state=42,svd_solver='randomized')
svc = SVC(kernel='rbf', class_weight='balanced',C=5,gamma=0.005)
gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.5, max_features=2, max_depth = 2, random_state = 0)
    constant_filter = StandardScaler()
model = make_pipeline(constant_filter,pca, gb)
%time model.fit(X_train, y_train)

In [ ]:
X_test=readImages('./bonus-sml-2020/SML_Test/')
X_test=np.asarray(X_test)
print("No of images:",X_test.shape)

In [ ]:
model = grid.best_estimator_
y_test = model.predict(X_test)

In [ ]:
def helper(direc): #helper fucntion for reading test file names
    files=[]
    for filename in natsorted(listdir(direc)):
        files.append(filename)
    return files

files=helper('./bonus-sml-2020/SML_Test/')
data_res=pd.DataFrame()
data_res['id']=files
data_res['category']=y_test
data_res.to_csv (r'sample_submission.csv', index = False, header=True)

In [ ]:
# model 1
# GridSearchCV(cv='warn', error_score='raise-deprecating',
#              estimator=Pipeline(memory=None,
#                                  steps=[('standardscaler',
#                                          StandardScaler(copy=True, with_mean=True, with_std=True)),
#                                         ('pca',
#                                          PCA(copy=True, iterated_power='auto', n_components=100,
#                                              random_state=42, svd_solver='randomized', tol=0.0,
#                                              whiten=True)),
#                                         ('mlpclassifier',
#                                          MLPClassifier(activation='tanh', alpha=1e-08,
#                                                        batch_size='auto', beta_1=0.9, beta_2=0.999,
#                                                        early_stopping=False, epsilon=1e-08,
#                                                        hidden_layer_sizes=(32, 32, 32),
#                                                        learning_rate='constant',
#                                                        learning_rate_init=0.001, max_fun=15000,
#                                                        max_iter=200, momentum=0.9, n_iter_no_change=10,
#                                                        nesterovs_momentum=True, power_t=0.5,
#                                                        random_state=0, shuffle=True, solver='adam',
#                                                        tol=0.0001, validation_fraction=0.1,
#                                                        verbose=False, warm_start=False))]),
#              iid='warn', n_jobs=None,
#              param_grid={'svc__C': [1, 5, 10, 50],
#                          'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]},
#              pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
#              scoring=None, verbose=0)

#model 2

# Pipeline(memory=None,
#          steps=[('standardscaler',
#                  StandardScaler(copy=True, with_mean=True, with_std=True)),
#                 ('pca',
#                  PCA(copy=True, iterated_power='auto', n_components=100,
#                      random_state=42, svd_solver='randomized', tol=0.0,
#                      whiten=True)),
#                 ('baggingclassifier',
#                  BaggingClassifier(base_estimator=SVC(C=5, break_ties=False,
#                                                       cache_size=200,
#                                                       class_weight='balanced',
#                                                       coef0=0.0,
#                                                       decision_function_shape='ovr',
#                                                       degree=3, gamma=0.005,
#                                                       kernel='rbf', max_iter=-1,
#                                                       probability=False,
#                                                       random_state=None,
#                                                       shrinking=True, tol=0.001,
#                                                       verbose=False),
#                                    bootstrap=True, bootstrap_features=False,
#                                    max_features=1.0, max_samples=1.0,
#                                    n_estimators=31, n_jobs=None,
#                                    oob_score=False, random_state=314, verbose=0,
#                                    warm_start=False))],
#          verbose=False)

# model 3 
# Pipeline(memory=None,
#          steps=[('standardscaler',
#                  StandardScaler(copy=True, with_mean=True, with_std=True)),
#                 ('pca',
#                  PCA(copy=True, iterated_power='auto', n_components=100,
#                      random_state=42, svd_solver='randomized', tol=0.0,
#                      whiten=True)),
#                 ('mlpclassifier',
#                  MLPClassifier(activation='tanh', alpha=1e-08,
#                                batch_size='auto', beta_1=0.9, beta_2=0.999,
#                                early_stopping=False, epsilon=1e-08,
#                                hidden_layer_sizes=(32, 32, 32),
#                                learning_rate='constant',
#                                learning_rate_init=0.001, max_fun=15000,
#                                max_iter=200, momentum=0.9, n_iter_no_change=10,
#                                nesterovs_momentum=True, power_t=0.5,
#                                random_state=0, shuffle=True, solver='adam',
#                                tol=0.0001, validation_fraction=0.1,
#                                verbose=False, warm_start=False))],
#          verbose=False)

# model 4
# Pipeline(memory=None,
#          steps=[('standardscaler',
#                  StandardScaler(copy=True, with_mean=True, with_std=True)),
#                 ('pca',
#                  PCA(copy=True, iterated_power='auto', n_components=100,
#                      random_state=42, svd_solver='randomized', tol=0.0,
#                      whiten=True)),
#                 ('gradientboostingclassifier',
#                  GradientBoostingClassifier(ccp_alpha=0.0,
#                                             criterion='friedman_mse', init=None,
#                                             learning_rate=0.5, loss='deviance',
#                                             max_depth=2, max_features=2,
#                                             max_leaf_nodes=None,
#                                             min_impurity_decrease=0.0,
#                                             min_impurity_split=None,
#                                             min_samples_leaf=1,
#                                             min_samples_split=2,
#                                             min_weight_fraction_leaf=0.0,
#                                             n_estimators=20,
#                                             n_iter_no_change=None,
#                                             presort='deprecated',
#                                             random_state=0, subsample=1.0,
#                                             tol=0.0001, validation_fraction=0.1,
#                                             verbose=0, warm_start=False))],
#          verbose=False)